# 6. Regression

### 6.1.Health Expenditure and Employment

In [43]:
# --- 데이터 로드 ---
metadata = pd.read_excel("/Users/yujin.sophia.kim/Desktop/data/WDI/archive/WDI_Indicators_Metadata.xlsx")
df = pd.read_csv("/Users/yujin.sophia.kim/Desktop/data/WDI/archive/WDI_Indicators_MainData.csv")

In [44]:
# --- 선택 & 리네임 (CHE 추가, 콤마 고침) ---
df = df[[
    "Country Name", "Time",
    "Current health expenditure (% of GDP) [SH.XPD.CHEX.GD.ZS]",
    "Domestic general government health expenditure (% of GDP) [SH.XPD.GHED.GD.ZS]",
    "Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]",
    "Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate) [SL.UEM.1524.ZS]",
    "Vulnerable employment, total (% of total employment) (modeled ILO estimate) [SL.EMP.VULN.ZS]", 
    "Part time employment, total (% of total employment) [SL.TLF.PART.ZS]",
    "Self-employed, total (% of total employment) (modeled ILO estimate) [SL.EMP.SELF.ZS]",
    "Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]"
]].copy()

df = df.rename(columns={
    "Current health expenditure (% of GDP) [SH.XPD.CHEX.GD.ZS]": "GDP_Total_HealthExp",
    "Domestic general government health expenditure (% of GDP) [SH.XPD.GHED.GD.ZS]": "GDP_GovHealthExp",
    "Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]": "JOB_Unemp_Total",
    "Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate) [SL.UEM.1524.ZS]": "JOB_Unemp_Youth_Total",
    "Vulnerable employment, total (% of total employment) (modeled ILO estimate) [SL.EMP.VULN.ZS]": "JOB_VulnerableEmployment_Total", 
    "Part time employment, total (% of total employment) [SL.TLF.PART.ZS]": "JOB_Part_Time_Total",
    "Self-employed, total (% of total employment) (modeled ILO estimate) [SL.EMP.SELF.ZS]": "JOB_Self_employed_Total",
    "Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]": "JOB_Contracters_Total",
})

# --- 숫자형 변환 ---
for col in ["GDP_Total_HealthExp","GDP_GovHealthExp","JOB_Unemp_Total","JOB_Unemp_Youth_Total",
            "JOB_VulnerableEmployment_Total","JOB_Part_Time_Total","JOB_Self_employed_Total","JOB_Contracters_Total"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df["Time"] = pd.to_numeric(df["Time"], errors="coerce").astype("Int64")

# --- 정부보건지출의 CHE 대비 비중(%) ---
df["GovShare_CHE_pct"] = 100 * df["GDP_GovHealthExp"] / df["GDP_Total_HealthExp"]

# --- 산점도용 집계 (국가별 평균) ---
govshare_avg = (df.groupby("Country Name", as_index=False)[
    ["GDP_GovHealthExp","JOB_Unemp_Total","JOB_Unemp_Youth_Total", "JOB_Self_employed_Total",
     "JOB_VulnerableEmployment_Total","GovShare_CHE_pct"]
].mean())

In [45]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS

entity_col = "Country Code" if "Country Code" in df.columns else "Country Name"
y_col = "GDP_GovHealthExp"

# 1) 사용할 설명변수 세트
X_all = [
    "JOB_Unemp_Total",
    "JOB_Unemp_Youth_Total",
    "JOB_VulnerableEmployment_Total",
    "JOB_Part_Time_Total",
    "JOB_Self_employed_Total",
    "JOB_Contracters_Total",
]
X_present = [c for c in X_all if c in df.columns]

# 기본 표본
use_cols = [entity_col, "Time", y_col] + X_present
d0 = (df.dropna(subset=[entity_col, "Time", y_col])
        .loc[:, use_cols].copy())
d0["Time"] = pd.to_numeric(d0["Time"], errors="coerce").astype(int)
d0 = (d0.sort_values([entity_col, "Time"])
         .drop_duplicates(subset=[entity_col, "Time"], keep="first"))

panel0 = d0.set_index([entity_col, "Time"]).sort_index().dropna()
y0 = pd.to_numeric(panel0[y_col], errors="coerce")
X0 = panel0[X_present].apply(pd.to_numeric, errors="coerce")

print(f"[Sample] N={len(panel0)}, entities={panel0.index.get_level_values(0).nunique()}, periods={panel0.index.get_level_values(1).nunique()}")
print("[Vars] ", list(X0.columns))

# ---------- A) 슬림 사양 (권장) ----------
keep = [c for c in ["JOB_Unemp_Total", "JOB_VulnerableEmployment_Total", "JOB_Part_Time_Total"] if c in X0.columns]
mod_slim = PanelOLS(y0, X0[keep], entity_effects=True, time_effects=True)

# (A-1) 국가만 클러스터
res_slim_ent = mod_slim.fit(cov_type="clustered", cluster_entity=True)
print("\n===== Slim spec (entity clustered) =====")
print(res_slim_ent.summary)

# (A-2) Driscoll–Kraay (공간상관·이분산·직렬상관 강건)
res_slim_dk = mod_slim.fit(cov_type="kernel", kernel="bartlett", bandwidth=3)  # bandwidth는 2~4 정도 시도
print("\n===== Slim spec (Driscoll–Kraay) =====")
print(res_slim_dk.summary)

# ---------- B) 표준화 베타 (해석 편의) ----------
Xz = X0[keep].apply(lambda s: (s - s.mean())/s.std(ddof=0))
yz = (y0 - y0.mean())/y0.std(ddof=0)
res_slim_std = PanelOLS(yz, Xz, entity_effects=True, time_effects=True)\
                 .fit(cov_type="kernel", kernel="bartlett", bandwidth=3)
print("\n===== Slim spec (standardized betas, DK SE) =====")
print(res_slim_std.params.sort_values())


[Sample] N=189, entities=18, periods=11
[Vars]  ['JOB_Unemp_Total', 'JOB_Unemp_Youth_Total', 'JOB_VulnerableEmployment_Total', 'JOB_Part_Time_Total', 'JOB_Self_employed_Total', 'JOB_Contracters_Total']

===== Slim spec (entity clustered) =====
                          PanelOLS Estimation Summary                           
Dep. Variable:       GDP_GovHealthExp   R-squared:                        0.0959
Estimator:                   PanelOLS   R-squared (Between):              0.0533
No. Observations:                 189   R-squared (Within):               0.1142
Date:                Fri, Oct 03 2025   R-squared (Overall):              0.0534
Time:                        15:33:37   Log-likelihood                   -48.875
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      5.5872
Entities:                          18   P-value                           0.0011
Avg Obs:                   

In [46]:
# --- 기본 식별자/시간/종속변수 ---
entity_col = "Country Code" if "Country Code" in df.columns else "Country Name"
y_col = "GDP_GovHealthExp"

# --- 후보 중 결측이 적은 쪽 하나만 선택 ---
candidates = [c for c in ["JOB_Contracters_Total", "JOB_Self_employed_Total"] if c in df.columns]
if candidates:
    chosen_status = max(candidates, key=lambda c: df[c].notna().sum())
else:
    chosen_status = None  # 둘 다 없으면 생략

# --- 최종 설명변수 세트(있는 것만) ---
X_base = ["JOB_Unemp_Total", "JOB_VulnerableEmployment_Total", "JOB_Part_Time_Total"]
X_vars = [c for c in X_base if c in df.columns]
if chosen_status:
    X_vars.append(chosen_status)

print("[Using regressors]", X_vars)

# --- 표본 구성 ---
use_cols = [entity_col, "Time", y_col] + X_vars
d = (df.dropna(subset=[entity_col, "Time", y_col])
       .loc[:, use_cols].copy())
d["Time"] = pd.to_numeric(d["Time"], errors="coerce").astype(int)
d = (d.sort_values([entity_col, "Time"])
       .drop_duplicates(subset=[entity_col, "Time"], keep="first"))

panel = d.set_index([entity_col, "Time"]).sort_index().dropna()
y = panel[y_col]
X = panel[X_vars]

print(f"[Sample] N={len(panel)}, entities={panel.index.get_level_values(0).nunique()}, periods={panel.index.get_level_values(1).nunique()}")

# --- 2-way FE + (권장) Driscoll–Kraay 표준오차 ---
mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
res = mod.fit(cov_type="kernel", kernel="bartlett", bandwidth=3)
print("\n===== Two-way FE (DK SE) =====")
print(res.summary)

# (선택) 표준화 베타
# Xz = (X - X.mean())/X.std(ddof=0); yz = (y - y.mean())/y.std(ddof=0)
# print(PanelOLS(yz, Xz, entity_effects=True, time_effects=True)
#       .fit(cov_type='kernel', kernel='bartlett', bandwidth=3).params.sort_values())


[Using regressors] ['JOB_Unemp_Total', 'JOB_VulnerableEmployment_Total', 'JOB_Part_Time_Total', 'JOB_Contracters_Total']
[Sample] N=189, entities=18, periods=11

===== Two-way FE (DK SE) =====
                          PanelOLS Estimation Summary                           
Dep. Variable:       GDP_GovHealthExp   R-squared:                        0.1206
Estimator:                   PanelOLS   R-squared (Between):             -2.7165
No. Observations:                 189   R-squared (Within):               0.1359
Date:                Fri, Oct 03 2025   R-squared (Overall):             -2.4427
Time:                        15:33:37   Log-likelihood                   -46.258
Cov. Estimator:        Driscoll-Kraay                                           
                                        F-statistic:                      5.3832
Entities:                          18   P-value                           0.0004
Avg Obs:                       10.500   Distribution:                   F(4,15

### 6.2. Education and Health Expenditure

In [47]:
# --- 데이터 로드 ---
metadata = pd.read_excel("/Users/yujin.sophia.kim/Desktop/data/WDI/archive/WDI_Indicators_Metadata.xlsx")
df = pd.read_csv("/Users/yujin.sophia.kim/Desktop/data/WDI/archive/WDI_Indicators_MainData.csv")

In [48]:
# 1) 식별자/시간
id_col  = "Country Name" if "Country Name" in df.columns else "Country Code"
time_col = "Time"         if "Time" in df.columns else "Year"

# 2) 리네임: 있으면만 바꾸기(errors='ignore')
rename_map = {
    "Current health expenditure (% of GDP) [SH.XPD.CHEX.GD.ZS]": "GDP_Total_HealthExp",
    "Domestic general government health expenditure (% of GDP) [SH.XPD.GHED.GD.ZS]": "GDP_GovHealthExp",
    "Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.BA.ZS]": "EDU_Bachelors_25Plus_Total",
    "Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.LO.ZS]": "EDU_LowerSecondary_25Plus_Total",
    "Educational attainment, at least completed post-secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.PO.ZS]": "EDU_PostSecondary_25Plus_Total",
    "Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative) [SE.PRM.CUAT.ZS]": "EDU_Primary_25Plus_Total",
    "Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative) [SE.TER.CUAT.ST.ZS]": "EDU_ShortCycleTertiary_25Plus_Total",
}
df = df.rename(columns=rename_map, errors="ignore")

# 3) 별칭 기준으로 필요한 컬럼만 선택 (있는 것만)
want = [
    id_col, time_col,
    "GDP_Total_HealthExp", "GDP_GovHealthExp",
    "EDU_Bachelors_25Plus_Total", "EDU_LowerSecondary_25Plus_Total",
    "EDU_PostSecondary_25Plus_Total", "EDU_Primary_25Plus_Total",
    "EDU_ShortCycleTertiary_25Plus_Total",
]
keep = [c for c in want if c in df.columns]
missing = [c for c in want if c not in df.columns]
df = df.loc[:, keep].copy()
print("[선택됨]", keep)
if missing: print("[없음]", missing)

# 4) 숫자형 변환
num_cols = [c for c in keep if c not in (id_col, time_col)]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df[time_col] = pd.to_numeric(df[time_col], errors="coerce").astype("Int64")


[선택됨] ['Country Name', 'Time', 'GDP_Total_HealthExp', 'GDP_GovHealthExp', 'EDU_Bachelors_25Plus_Total', 'EDU_LowerSecondary_25Plus_Total', 'EDU_PostSecondary_25Plus_Total', 'EDU_Primary_25Plus_Total', 'EDU_ShortCycleTertiary_25Plus_Total']


In [49]:
# === Education composition -> Gov health expenditure FE regression ===
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS

# 설정
YR0, YR1 = 2011, 2021
entity_col = "Country Code" if "Country Code" in df.columns else "Country Name"
time_col   = "Time" if "Time" in df.columns else "Year"
y_col = "GDP_GovHealthExp"   # 종속변수

# 누적(≥) 지표 이름 (이미 리네임된 별칭 기준)
BAp  = "EDU_Bachelors_25Plus_Total"          # ≥ ISCED 6
SCtp = "EDU_ShortCycleTertiary_25Plus_Total" # ≥ ISCED 5
PSnp = "EDU_PostSecondary_25Plus_Total"      # ≥ ISCED 4
LSp  = "EDU_LowerSecondary_25Plus_Total"     # ≥ ISCED 2
PRMp = "EDU_Primary_25Plus_Total"            # ≥ ISCED 1
need = [BAp, SCtp, PSnp, LSp, PRMp]
missing = [c for c in need if c not in df.columns]
if missing:
    raise KeyError(f"교육 누적 지표가 부족합니다: {missing}")

# 숫자형/기간 필터
d = df.copy()
d[time_col] = pd.to_numeric(d[time_col], errors="coerce").astype("Int64")
for c in need + [y_col]:
    d[c] = pd.to_numeric(d[c], errors="coerce")
d = d[d[time_col].between(YR0, YR1)].copy()

# 단조성 보정(≥1 ≥2 ≥4 ≥5 ≥6)
d["_P1"] = d[PRMp]
d["_P2"] = np.minimum(d[LSp],  d["_P1"])
d["_P4"] = np.minimum(d[PSnp], d["_P2"])
d["_P5"] = np.minimum(d[SCtp], d["_P4"])
d["_P6"] = np.minimum(d[BAp],  d["_P5"])

# 배타 카테고리(합≈100), 기준집단은 BelowPrimary로 둘 것
d["EXC_BAplus"]             = d["_P6"]
d["EXC_ShortCycle_only"]    = (d["_P5"] - d["_P6"]).clip(0, 100)
d["EXC_PostSecondary_only"] = (d["_P4"] - d["_P5"]).clip(0, 100)
d["EXC_Lower_or_UpperSec"]  = (d["_P2"] - d["_P4"]).clip(0, 100)
d["EXC_Primary_only"]       = (d["_P1"] - d["_P2"]).clip(0, 100)
d["EXC_BelowPrimary"]       = (100 - d["_P1"]).clip(0, 100)

X_edu = [
    "EXC_Primary_only",
    "EXC_Lower_or_UpperSec",
    "EXC_PostSecondary_only",
    "EXC_ShortCycle_only",
    "EXC_BAplus",            # <-- BelowPrimary는 baseline으로 제외(완전공선성 회피)
]

# 패널 구성
use_cols = [entity_col, time_col, y_col] + X_edu
d = (d.dropna(subset=[entity_col, time_col, y_col])
       .loc[:, use_cols]
       .sort_values([entity_col, time_col])
       .drop_duplicates(subset=[entity_col, time_col], keep="first"))
panel = d.set_index([entity_col, time_col]).sort_index().dropna()

y = panel[y_col]
X = panel[X_edu]

print(f"[Sample] N={len(panel)}, entities={panel.index.get_level_values(0).nunique()}, "
      f"periods={panel.index.get_level_values(1).nunique()}")

# 2-way FE + Driscoll–Kraay SE (권장)
mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
res = mod.fit(cov_type="kernel", kernel="bartlett", bandwidth=3)
print("\n===== FE (DK SE): GovHealthExp ~ Education composition (baseline = Below Primary) =====")
print(res.summary)

# 표준화 베타(해석 편의)
Xz = (X - X.mean())/X.std(ddof=0)
yz = (y - y.mean())/y.std(ddof=0)
res_std = PanelOLS(yz, Xz, entity_effects=True, time_effects=True)\
            .fit(cov_type="kernel", kernel="bartlett", bandwidth=3)
print("\nStandardized betas (DK):")
print(res_std.params.sort_values().round(3))


[Sample] N=166, entities=18, periods=11

===== FE (DK SE): GovHealthExp ~ Education composition (baseline = Below Primary) =====
                          PanelOLS Estimation Summary                           
Dep. Variable:       GDP_GovHealthExp   R-squared:                        0.0496
Estimator:                   PanelOLS   R-squared (Between):             -0.2256
No. Observations:                 166   R-squared (Within):              -0.2952
Date:                Fri, Oct 03 2025   R-squared (Overall):             -0.2160
Time:                        15:33:38   Log-likelihood                   -28.222
Cov. Estimator:        Driscoll-Kraay                                           
                                        F-statistic:                      1.3883
Entities:                          18   P-value                           0.2327
Avg Obs:                       9.2222   Distribution:                   F(5,133)
Min Obs:                       3.0000                        

In [50]:
# ================================================
# FE regression: GovHealthExp ~ Education (blocks)
# - 블록 단순화, DK 표준오차, 선택적 통제, 시차(1년)
# ================================================
import re, numpy as np, pandas as pd
from linearmodels.panel import PanelOLS

# ------------------------
# 0) 기본 설정
# ------------------------
YR0, YR1 = 2011, 2021
entity_col = "Country Code" if "Country Code" in df.columns else "Country Name"
time_col   = "Time" if "Time" in df.columns else "Year"
y_col      = "GDP_GovHealthExp"

# ------------------------
# 1) EXC_* 없으면 EDU_*에서 배타 카테고리 계산
# ------------------------
def ensure_exclusive_categories(df_in: pd.DataFrame) -> pd.DataFrame:
    d = df_in.copy()
    # 이미 EXC_*가 있으면 그대로 사용
    exc_exist = [c for c in d.columns if str(c).startswith("EXC_")]
    if len(exc_exist) >= 4:
        return d

    # 누적(≥) 지표 별칭
    BAp  = "EDU_Bachelors_25Plus_Total"          # ≥ ISCED 6
    SCtp = "EDU_ShortCycleTertiary_25Plus_Total" # ≥ ISCED 5
    PSnp = "EDU_PostSecondary_25Plus_Total"      # ≥ ISCED 4
    LSp  = "EDU_LowerSecondary_25Plus_Total"     # ≥ ISCED 2
    PRMp = "EDU_Primary_25Plus_Total"            # ≥ ISCED 1
    need = [BAp, SCtp, PSnp, LSp, PRMp]

    # code/원문/별칭 자동 탐색
    code_map = {
        BAp:  "SE.TER.CUAT.BA.ZS",
        SCtp: "SE.TER.CUAT.ST.ZS",
        PSnp: "SE.SEC.CUAT.PO.ZS",
        LSp:  "SE.SEC.CUAT.LO.ZS",
        PRMp: "SE.PRM.CUAT.ZS",
    }
    def find_col(name, code):
        if name in d.columns: return name
        hit = [c for c in d.columns if code in str(c)]
        if hit: return hit[0]
        head = name.split("[")[0].strip()
        for c in d.columns:
            if head and head.lower() in str(c).lower():
                return c
        return None

    src = {}
    for alias, code in code_map.items():
        col = find_col(alias, code)
        if col is None:
            raise KeyError(f"EDU 누적 지표가 부족합니다: {alias} / {code}")
        src[alias] = col

    # 숫자형+클리핑
    for alias, col in src.items():
        d[alias] = pd.to_numeric(d[col], errors="coerce").clip(0, 100)

    # 단조성 보정(≥1 ≥2 ≥4 ≥5 ≥6)
    d["_P1"] = d[PRMp]
    d["_P2"] = np.minimum(d[LSp],  d["_P1"])
    d["_P4"] = np.minimum(d[PSnp], d["_P2"])
    d["_P5"] = np.minimum(d[SCtp], d["_P4"])
    d["_P6"] = np.minimum(d[BAp],  d["_P5"])

    # 배타 카테고리 (합≈100)
    d["EXC_BAplus"]             = d["_P6"]
    d["EXC_ShortCycle_only"]    = (d["_P5"] - d["_P6"]).clip(0, 100)
    d["EXC_PostSecondary_only"] = (d["_P4"] - d["_P5"]).clip(0, 100)  # ISCED 4 (비대학)
    d["EXC_Lower_or_UpperSec"]  = (d["_P2"] - d["_P4"]).clip(0, 100)  # ISCED 2–3
    d["EXC_Primary_only"]       = (d["_P1"] - d["_P2"]).clip(0, 100)  # ISCED 1
    d["EXC_BelowPrimary"]       = (100 - d["_P1"]).clip(0, 100)       # ISCED 0-

    d.drop(columns=["_P1","_P2","_P4","_P5","_P6"], inplace=True)
    return d

d0 = ensure_exclusive_categories(df).copy()

# ------------------------
# 2) 블록 단순화 (해석 쉬운 버전)
#    - Tertiary = ISCED 5–8 = ShortCycle + BAplus
#    - PostSecNonTer = ISCED 4
#    - Secondary = ISCED 2–3
#    - PrimaryOrBelow = ISCED 0–1  (baseline)
# ------------------------
d0["EDU_Tertiary"]       = pd.to_numeric(d0["EXC_ShortCycle_only"], errors="coerce") + \
                           pd.to_numeric(d0["EXC_BAplus"], errors="coerce")
d0["EDU_PostSecNonTer"]  = pd.to_numeric(d0["EXC_PostSecondary_only"], errors="coerce")
d0["EDU_Secondary"]      = pd.to_numeric(d0["EXC_Lower_or_UpperSec"], errors="coerce")
d0["EDU_PrimaryOrBelow"] = pd.to_numeric(d0["EXC_Primary_only"], errors="coerce") + \
                           pd.to_numeric(d0["EXC_BelowPrimary"], errors="coerce")

# ------------------------
# 3) 선택적 통제 변수 자동 탐색 (있으면 사용)
#    - log GDP per capita (NY.GDP.PCAP.*)
#    - 65+ (% of total) (SP.POP.65UP.TO.ZS)
#    - Unemployment total (SL.UEM.TOTL.ZS) 또는 별칭
# ------------------------
ctrls = []

def pick_control(codes, keywords):
    for c in d0.columns:
        s = str(c)
        if any(code in s for code in codes) or any(k.lower() in s.lower() for k in keywords):
            return c
    return None

col_gdppc = pick_control(["NY.GDP.PCAP"], ["GDP per capita"])
if col_gdppc is not None:
    d0["CTRL_log_gdppc"] = np.log(pd.to_numeric(d0[col_gdppc], errors="coerce"))
    ctrls.append("CTRL_log_gdppc")

col_65p = pick_control(["SP.POP.65UP.TO.ZS"], ["ages 65"])
if col_65p is not None:
    d0["CTRL_65plus"] = pd.to_numeric(d0[col_65p], errors="coerce")
    ctrls.append("CTRL_65plus")

if "JOB_Unemp_Total" in d0.columns:
    d0["CTRL_Unemp"] = pd.to_numeric(d0["JOB_Unemp_Total"], errors="coerce")
elif pick_control(["SL.UEM.TOTL.ZS"], ["Unemployment, total"]) is not None:
    col_u = pick_control(["SL.UEM.TOTL.ZS"], ["Unemployment, total"])
    d0["CTRL_Unemp"] = pd.to_numeric(d0[col_u], errors="coerce")
if "CTRL_Unemp" in d0.columns:
    ctrls.append("CTRL_Unemp")

print("[Controls used]", ctrls if ctrls else "None")

# ------------------------
# 4) 패널 정리 + 시차(1년) 옵션
# ------------------------
d0[time_col] = pd.to_numeric(d0[time_col], errors="coerce").astype("Int64")
d0 = d0[d0[time_col].between(YR0, YR1)].copy()
d0[y_col] = pd.to_numeric(d0[y_col], errors="coerce")

# 설명변수 세트 (baseline = PrimaryOrBelow)
X_blocks = ["EDU_Secondary", "EDU_PostSecNonTer", "EDU_Tertiary"] + ctrls

# 1년 시차 버전 만들기 (교육 블록만 lag)
d0 = d0.sort_values([entity_col, time_col])
for c in ["EDU_Secondary","EDU_PostSecNonTer","EDU_Tertiary"]:
    d0[c+"_L1"] = d0.groupby(entity_col)[c].shift(1)

X_blocks_lag = [c+"_L1" for c in ["EDU_Secondary","EDU_PostSecNonTer","EDU_Tertiary"]] + ctrls

# ------------------------
# 5) FE 추정 (동일 국가/연도 FE, DK SE)
# ------------------------
def run_fe(df_in, Xlist, label):
    use = [entity_col, time_col, y_col] + Xlist
    dd = (df_in
          .dropna(subset=[entity_col, time_col, y_col])
          .loc[:, [c for c in use if c in df_in.columns]]
          .dropna()
          .set_index([entity_col, time_col])
          .sort_index())
    y = dd[y_col]
    X = dd[[c for c in Xlist if c in dd.columns]]
    mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
    res = mod.fit(cov_type="kernel", kernel="bartlett", bandwidth=3)
    print(f"\n===== FE (DK SE) — {label} =====")
    print(f"[Sample] N={len(dd)}, entities={dd.index.get_level_values(0).nunique()}, periods={dd.index.get_level_values(1).nunique()}")
    print(res.summary)

    # 표준화 베타
    Xz = (X - X.mean())/X.std(ddof=0)
    yz = (y - y.mean())/y.std(ddof=0)
    res_std = PanelOLS(yz, Xz, entity_effects=True, time_effects=True)\
                .fit(cov_type="kernel", kernel="bartlett", bandwidth=3)
    print("\nStandardized betas (DK):")
    print(res_std.params.sort_values().round(3))
    return res, res_std

# (A) 동기 변수
res_now, res_now_std = run_fe(d0, X_blocks, "Education blocks (contemporaneous)")

# (B) 1년 시차 변수
res_lag, res_lag_std = run_fe(d0, X_blocks_lag, "Education blocks (lagged 1y)")


[Controls used] None

===== FE (DK SE) — Education blocks (contemporaneous) =====
[Sample] N=166, entities=18, periods=11
                          PanelOLS Estimation Summary                           
Dep. Variable:       GDP_GovHealthExp   R-squared:                        0.0421
Estimator:                   PanelOLS   R-squared (Between):             -0.6298
No. Observations:                 166   R-squared (Within):              -0.2374
Date:                Fri, Oct 03 2025   R-squared (Overall):             -0.6164
Time:                        15:33:38   Log-likelihood                   -28.878
Cov. Estimator:        Driscoll-Kraay                                           
                                        F-statistic:                      1.9761
Entities:                          18   P-value                           0.1205
Avg Obs:                       9.2222   Distribution:                   F(3,135)
Min Obs:                       3.0000                               